In [ ]:
import pandas as pd
from ssh_session import get_ssh_client

import panel as pn
import time
pn.extension()
pn.extension('tabulator')
pn.extension(notifications=True)
pn.extension('jsoneditor')
import os
import configparser
from io import StringIO
import subprocess


## Set SSH Connection

In [ ]:

ssh_server_name = os.environ["HPC_ID"]  # Change to your server name if needed


ssh = get_ssh_client(ssh_server_name)

stdin, stdout, stderr = ssh.exec_command("hostname")
print("Remote host says:", stdout.read().decode().strip())

# Call again: reuses existing connection
ssh2 = get_ssh_client(ssh_server_name)
print("Same connection reused?", ssh is ssh2)



## Job List

In [ ]:

config_path = os.path.expanduser('~/ssh_config.ini')
with open(config_path, 'r') as f:
    config_content = f.read()

config = configparser.ConfigParser()
config.read_string(config_content)

data = []
for section in config.sections():
    row = {'Job Name': section}
    for key, value in config.items(section):
        row[key] = value
    data.append(row)

config_df = pd.DataFrame(data)


df_pane_job_list = pn.widgets.Tabulator(
    config_df,
    show_index=False,
    width=800, height=320,
    sizing_mode="stretch_width",
    layout="fit_data_table",
    buttons={'Run': '▶️ Run'},   # Button per row
)

job_list = pn.Column(df_pane_job_list, width=1400, height=400)


## Running Jobs

In [ ]:
df_pane_running_jobs = pn.widgets.Tabulator(
    pd.DataFrame(),
    show_index=False,
    width=800, height=320,
    sizing_mode="stretch_width",
    layout="fit_data_table",
    buttons={'Delete': '🗑 Delete'},   # Button per row
)

running_jobs = pn.Column(df_pane_running_jobs, width=950, height=500)

## Experiment Editor

In [ ]:
experiment_name = ""
json_file = os.path.expanduser(f"~/.maia-hpc/experiments/{experiment_name}.json")

import json
# Load JSON file
def load_json():
    try:
        with open(json_file, "r") as f:
            return json.load(f)
    except Exception as e:
        return {"error": str(e)}

# Save JSON file
def save_json(event):
    try:
        content = editor.value
        parsed = json.loads(content)  # validate JSON
        with open(json_file, "w") as f:
            json.dump(parsed, f, indent=2)
        status.value = "✅ Saved successfully."
    except Exception as e:
        status.value = f"❌ Error: {e}"

editor = pn.widgets.JSONEditor(
    value=load_json(),
    mode="tree",   # options: "tree", "code", "form", "text", "view"
    height=500,
    width=600
)

# Button to print/save updated JSON

save_button = pn.widgets.Button(name="Save", button_type="primary")
save_button.on_click(save_json)

app = pn.Column(
    "# JSON Editor (Form Mode)",
    editor,
    save_button
)

## Logs

In [ ]:
def get_slurm_log():
    try:
        selected_indexes = df_pane_running_jobs.selection  # <-- get selected row indices
        if selected_indexes:
            # Get the first selected row
            row_index = selected_indexes[0]
            job_id = str(df_pane_running_jobs.value.iloc[row_index]["JOBID"])
            log_path = f"logs/slurm-{job_id}.out"
            stdin, stdout, stderr = ssh.exec_command(f"cat {log_path}")
            output = stdout.read().decode(errors="replace")
            if output.strip() == "":
                return stderr.read().decode().strip() or "Log file is empty."
            return output
        else:
            return "No job selected."
    except Exception as e:
        return f"Error: {e}"

def get_slurm_log_err():
    try:
        selected_indexes = df_pane_running_jobs.selection  # <-- get selected row indices
        if selected_indexes:
            # Get the first selected row
            row_index = selected_indexes[0]
            job_id = str(df_pane_running_jobs.value.iloc[row_index]["JOBID"])
            log_path = f"logs/slurm-{job_id}.err"
            stdin, stdout, stderr = ssh.exec_command(f"cat {log_path}")
            output = stdout.read().decode(errors="replace")
            if output.strip() == "":
                return stderr.read().decode().strip() or "Error log file is empty."
            return output
        else:
            return "No job selected."
    except Exception as e:
        return f"Error: {e}"

# Fetch content
log_content = get_slurm_log()
log_err_content = get_slurm_log_err()


# TextAreaInput (read-only)
textarea = pn.widgets.TextAreaInput(
    name="Slurm Log Output",
    value=log_content,
    height=400,
    width=600,
    disabled=True  # read-only
)

textarea_err = pn.widgets.TextAreaInput(
    name="Slurm Error Output",
    value=log_err_content,
    height=400,
    width=600,
    disabled=True  # read-only
)

log_output = pn.Column(
    "# Slurm Log Viewer",
    textarea
)

log_output_err = pn.Column(
    "# Slurm Error Viewer",
    textarea_err
)


In [ ]:
pcb = None  # periodic callback reference

def autorefresh():
    stdin, stdout, stderr = ssh.exec_command(
        "squeue -o '%.18i %.40j %.8u %.2t %.10M %.6D %R' -u $USER"
    )
    output = stdout.read().decode().strip()
    lines = output.splitlines()

    output_sacct = ""
    try:
        stdin_sacct, stdout_sacct, stderr_sacct = ssh.exec_command(
            "sacct -u $USER --format=JobID,JobName,User,State,Elapsed,NodeList,Start -P --starttime=$(date -d '1 hour ago' +'%Y-%m-%dT%H:%M:%S')"
        )
        output_sacct = stdout_sacct.read().decode().strip()
    except Exception as e:
        print(f"Error fetching sacct output: {e}")

    sacct_lines = output_sacct.splitlines()

    if lines:
        header = lines[0].split()
        rows = [line.split(None, len(header)-1) for line in lines[1:]]
    else:
        df_pane_running_jobs.value = pd.DataFrame()

    if sacct_lines and len(sacct_lines) > 1:
        for line in sacct_lines[1:]:
          if "." not in line.split("|")[0]:
              row = line.split("|")
              #row.append(None)
              rows.append(row)

    df = pd.DataFrame(rows, columns=header)
    df_pane_running_jobs.value = df

    log_content = get_slurm_log()
    textarea.value = log_content

    log_err_content = get_slurm_log_err()
    textarea_err.value = log_err_content


def _delete_job(jobid: str):
    if pcb: pcb.stop()
    try:
        ssh.exec_command(f"scancel {jobid}")
        pn.state.notifications.info(f"Job {jobid} deleted", duration=3000)
        autorefresh()
    finally:
        if pcb: pcb.start()

# Ensure we delete only the clicked row’s job
def _on_click(event):
    if event.column == "Delete":
        row_idx = event.row
        current = df_pane_running_jobs.value
        if 0 <= row_idx < len(current):
            jobid = str(current.iloc[row_idx]["JOBID"])
            _delete_job(jobid)
    else:
        log_content = get_slurm_log()
        textarea.value = log_content
        log_err_content = get_slurm_log_err()
        textarea_err.value = log_err_content

df_pane_running_jobs.on_click(_on_click)
pcb = pn.state.add_periodic_callback(autorefresh, period=2000)

In [ ]:
def _run_job(job_name):
    print(f"Running job: {job_name}")
    subprocess.run(["submit_job.sh", job_name])
    pn.state.notifications.info(f"Job {job_name} submitted", duration=3000)

def _on_click_run(event):
    row = df_pane_job_list.value.iloc[event.row] if 0 <= event.row < len(df_pane_job_list.value) else None
    if row is not None:
        if event.column == 'Run':
            job_name = row['Job Name']
            _run_job(job_name)
            
def _on_click_display_experiment(event):
    row = df_pane_job_list.value.iloc[event.row] if 0 <= event.row < len(df_pane_job_list.value) else None
    if row is not None:
        if event.column == 'experiment_name':
           experiment_name = row['experiment_name']
           _display_experiment(experiment_name)

def _display_experiment(exp_name):
    print(f"Displaying experiment: {exp_name}")
    global experiment_name
    global json_file
    experiment_name = exp_name
    json_file = os.path.expanduser(f"~/.maia-hpc/experiments/{experiment_name}.json")
    editor.value = load_json()
    # Add code to display the experiment details

df_pane_job_list.on_click(_on_click_run, 'Run')
df_pane_job_list.on_click(_on_click_display_experiment, 'experiment_name')

In [ ]:
pn.template.BootstrapTemplate(
    title="MAIA-HPC",
    main=[
        pn.Card(running_jobs, title="Running Jobs", collapsible=True),
        pn.Card(job_list, title="Job List", collapsible=True),
        pn.Card(app, title="Job Editor", collapsible=True),
        pn.Row(
            pn.Card(log_output, title="Slurm Log Out Viewer", collapsible=True),
            pn.Card(log_output_err, title="Slurm Error Viewer", collapsible=True),
        ),
    ]
).servable()
